In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from IPython.display import display
import matplotlib.pyplot as plt
from keras.models import Model
from pathlib import Path
import tensorflow as tf
from time import sleep
from PIL import Image
import urllib.request
import numpy as np
import tweepy
import cv2
import os
import io

In [ ]:
cloud_cnn = tf.keras.models.load_model('/content/drive/MyDrive/CloudImages/vgg_model_ft.h5') #87% accurate weights

In [ ]:
def main():
    twitter_auth_keys = {
        "consumer_key"        : "consumer_key",
        "consumer_secret"     : "consumer_secret",
        "access_token"        : "access_token",
        "access_token_secret" : "access_token_secret"
    }
 
    auth = tweepy.OAuthHandler(
            twitter_auth_keys['consumer_key'],
            twitter_auth_keys['consumer_secret']
            )
    auth.set_access_token(
            twitter_auth_keys['access_token'],
            twitter_auth_keys['access_token_secret']
            )
    api = tweepy.API(auth)
 
    def predictCloudForm(filename):
      def loadWeatherFile(filename):
        url = "http://uindy-weathercam.s3.us-east-2.amazonaws.com/" + filename
        fh = urllib.request.urlopen(url)
        return Image.open(io.BytesIO(fh.read()))

      img_array = image.img_to_array(loadWeatherFile(filename).resize((224,224)))
      img_batch = np.expand_dims(img_array, axis=0)
      img_preprocessed = preprocess_input(img_batch)
      prediction = cloud_cnn.predict(img_preprocessed)
      vgg_pred_classes_ft = np.argmax(prediction, axis=1)

      if vgg_pred_classes_ft[0] == 0:
        cloud_form = "Cirrus"
      elif vgg_pred_classes_ft[0] == 1:
        cloud_form = "Cumulus"
      elif vgg_pred_classes_ft[0] == 2:
        cloud_form = "Other"
      elif vgg_pred_classes_ft[0] == 3:
        cloud_form = "Stratus"

      predList = np.array(prediction).tolist()
      confid = predList[0][vgg_pred_classes_ft[0]] * 100

      daily_image = loadWeatherFile(filename)
      daily_image.save('/content/drive/MyDrive/CloudImages/daily_image/' + filename)

      cloud_status = "I'm {}% confident the cloud formation is: {}".format(round(confid, 2), cloud_form)

      api.update_with_media('/content/drive/MyDrive/CloudImages/daily_image/' + filename, status=cloud_status)

    predictCloudForm('1594997760.jpg') #the .jpg here will be the latest image taken from the camera

     
if __name__ == "__main__":
    main()